Desired output posted in 'Group-Desired.xlxs'

In [1]:
import pandas as pd
import sqlite3 as db

In [3]:
group1_df = pd.read_csv(r'C:\Users\Arun\Downloads\JupyterNotebooks\Group-Wrangling\Group1.csv')
group2_df = pd.read_csv(r'C:\Users\Arun\Downloads\JupyterNotebooks\Group-Wrangling\Group2.csv')

In [2]:
conn = db.connect('sqliteprac.db')

In [5]:
group1_df.to_sql('group1', conn, if_exists='replace', index = False)
group2_df.to_sql('group2', conn, if_exists='replace', index = False)

7

In [3]:
query = 'select * from group1'
pd.read_sql_query(query,conn)

,PersonID,Height_CM,DOB
0,14521,160,03-09-1963
1,15321,171,12-02-1964
2,21821,175,11-10-1976
3,31721,169,12-02-1981
4,42421,167,12-03-1982
5,11431,163,12-04-1979
6,16123,162,10-06-1989


In [4]:
query = 'select * from group2'
pd.read_sql_query(query,conn)

,PersonID,Height_CM,DOB
0,14521,166,09-03-1991
1,15121,171,12-02-1989
2,21824,175,11-10-1993
3,39721,169,12-02-1981
4,42291,167,02-03-1982
5,11931,163,12-04-1984
6,14123,162,10-06-1986


In [5]:
#Find the datatype of DOB in group1 and group2
query = """
select DOB,
       typeof(DOB) as Type,
       'Group1' as Table_name
       from Group1        
union
select DOB,
       typeof(DOB) as Type,
       'Group2' as Table_name
       from Group2 
order by Table_name
"""
pd.read_sql_query(query,conn)

,DOB,Type,Table_name
0,03-09-1963,text,Group1
1,10-06-1989,text,Group1
2,11-10-1976,text,Group1
3,12-02-1964,text,Group1
4,12-02-1981,text,Group1
5,12-03-1982,text,Group1
6,12-04-1979,text,Group1
7,02-03-1982,text,Group2
8,09-03-1991,text,Group2
9,10-06-1986,text,Group2


In [6]:
#SQLite doesn't include a datatype called 'date', so re-arrange to yyyy-mm-dd format 
#8 temp tables - sep_dob1, con_dob1, sep_dob2, con_dob2, group1_eldest, group1_tallest,group2_eldest, group2_tallest 
#sep_dob1 and con_dob1 changes the date format in Group1
#sep_dob2 and con_dob2 changes the date format in Group2
#group1_eldest and group1_tallest finds the eldest and tallest from temp table con_dob1
#group2_eldest and group2_tallest finds the eldest and tallest from temp table con_dob2

query = """
with sep_dob1 as 
(
select *, 
       substr(DOB,7,4) as year,
       substr(DOB,4,2) as month,
       substr(DOB,1,2) as date
from group1
),

con_dob1 as 
(
select *, 
       year||'-'||month||'-'||date as upd_dob1
from sep_dob1
),

sep_dob2 as 
(
select *, 
       substr(DOB,7,4) as year,
       substr(DOB,4,2) as month,
       substr(DOB,1,2) as date
from group2
),

con_dob2 as 
(
select *,
       year||'-'||month||'-'||date as upd_dob2
from sep_dob2
),

group1_eldest as 
(
select min(upd_dob1) as BirthDate,
       'Group 1' as GroupName,
       PersonID,
       Height_CM as Height,
       'Eldest' as Remarks
from con_dob1
),

group1_tallest as
(
select upd_dob1 as BirthDate,
       'Group 1' as GroupName,
       PersonID,
       max(Height_CM) as Height,
       'Tallest' as Remarks
from con_dob1
),

group2_eldest as
(
select min(upd_dob2) as BirthDate,
       'Group 2' as GroupName,
       PersonID,
       Height_CM as Height,
       'Eldest' as Remarks
from con_dob2
),

group2_tallest as
(
select upd_dob2 as BirthDate,
       'Group 2' as GroupName,
       PersonID,
       max(Height_CM) as Height,
       'Tallest' as Remarks
from con_dob2
)

select * from group1_eldest
union
select * from group1_tallest
union
select * from group2_eldest
union
select * from group2_tallest
"""
pd.read_sql_query(query,conn)

,BirthDate,GroupName,PersonID,Height,Remarks
0,1963-09-03,Group 1,14521,160,Eldest
1,1976-10-11,Group 1,21821,175,Tallest
2,1981-02-12,Group 2,39721,169,Eldest
3,1993-10-11,Group 2,21824,175,Tallest


In [7]:
conn.commit()
conn.close()